## Face recognition test

Main portion of code obtained from [this notebook](https://github.com/timesler/facenet-pytorch/blob/master/examples/infer.ipynb)

## Setup

In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

In [3]:
workers = 0

In [4]:
device = torch.device('cpu')

## Load dataset

In [5]:
def collate_fn(x):
    return x[0]

data_set = datasets.ImageFolder('./slack_faces')
data_set.idx_to_class = {i:c for c, i in data_set.class_to_idx.items()}
data_loader = DataLoader(data_set, collate_fn=collate_fn, num_workers=workers)

## Use MTCNN for face recognition

In [6]:
mtcnn = MTCNN(device=device)

In [7]:
data_aligned = []
data_names = []
for x, y in data_loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        data_aligned.append(x_aligned)
        data_names.append(data_set.idx_to_class[y])

## Resnet is used for face embedding

In [8]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [9]:
data_aligned = torch.stack(data_aligned).to(device)
data_embeddings = resnet(data_aligned).detach().cpu()

In [10]:
dists = [[(e1 - e2).norm().item() for e2 in data_embeddings] for e1 in data_embeddings]
print(pd.DataFrame(dists, columns=data_names, index=data_names))

                          andres_cadiz  andres_matte  diego_andai  \
andres_cadiz                  0.000000      1.119540     1.267596   
andres_matte                  1.119540      0.000000     1.361286   
diego_andai                   1.267596      1.361286     0.000000   
diego_fernandez               1.314620      1.304183     1.316425   
ignacio_marquez               1.052495      1.166882     1.128461   
jaime_bunzli                  1.310948      1.326655     1.404345   
javier_montoya                1.296394      1.324368     1.308123   
maria_fernanda_sepulveda      1.269143      1.340715     1.379270   
milan_rafaeli                 1.476271      1.428223     1.371135   
raimundo_herrera              1.352822      1.417893     1.410501   

                          diego_fernandez  ignacio_marquez  jaime_bunzli  \
andres_cadiz                     1.314620         1.052495      1.310948   
andres_matte                     1.304183         1.166882      1.326655   
diego_andai 

## Load testset and recognize faces

In [11]:
def collate_fn(x):
    return x[0]

test_set = datasets.ImageFolder('./test_data')
test_set.idx_to_class = {i:c for c, i in test_set.class_to_idx.items()}
test_loader = DataLoader(test_set, collate_fn=collate_fn, num_workers=workers)

In [12]:
test_aligned = []
test_names = []
for x, y in test_loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        test_aligned.append(x_aligned)
        test_names.append(test_set.idx_to_class[y])

In [13]:
test_aligned = torch.stack(test_aligned).to(device)
test_embeddings = resnet(test_aligned).detach().cpu()

## KNN Test (nearest embedding)

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
neigh = KNeighborsClassifier(n_neighbors=1)

In [16]:
neigh.fit(data_embeddings, data_names)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [27]:
print(f"Accuracy: {(neigh.score(test_embeddings, test_names) * 100):.2f}%")

Accuracy: 100.00%


In [28]:
for idx, embedding in enumerate(test_embeddings):
    prediction = neigh.predict(embedding.reshape(1, -1))[0]
    print(f"for {test_names[idx]}, predicted: {prediction}")

for andres_cadiz, predicted: andres_cadiz
for andres_cadiz, predicted: andres_cadiz
for andres_matte, predicted: andres_matte
for andres_matte, predicted: andres_matte
for diego_andai, predicted: diego_andai
for diego_fernandez, predicted: diego_fernandez
for diego_fernandez, predicted: diego_fernandez
for ignacio_marquez, predicted: ignacio_marquez
for ignacio_marquez, predicted: ignacio_marquez
for jaime_bunzli, predicted: jaime_bunzli
for jaime_bunzli, predicted: jaime_bunzli
for javier_montoya, predicted: javier_montoya
for maria_fernanda_sepulveda, predicted: maria_fernanda_sepulveda
for milan_rafaeli, predicted: milan_rafaeli
for raimundo_herrera, predicted: raimundo_herrera
